In [1]:
from unsloth import FastModel,FastLanguageModel
import torch
from torch.nn.utils import clip_grad_value_
import torch.nn as nn
from peft import PeftModel
import gc
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=16,
)
base_model = model.base_model
base_model.lm_head.weight.requires_grad_(True);
gc.collect()
torch.cuda.empty_cache()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-05 11:56:23 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
base_model.lm_head.load_state_dict(torch.load('../Model/lm_heads_weights_pt.pth'))
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           # Larger = higher accuracy, but might overfit
    lora_alpha = 64,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)
# model = FastLanguageModel.for_inference(model)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [4]:
model.save_pretrained_merged??

Signature:
model.save_pretrained_merged(
    save_directory: Union[str, os.PathLike],
    tokenizer=None,
    save_method: str = 'merged_16bit',
    push_to_hub: bool = False,
    token: Union[bool, str, NoneType] = None,
    is_main_process: bool = True,
    state_dict: Optional[dict] = None,
    save_function: Callable = <function save at 0x7c4050483b00>,
    max_shard_size: Union[int, str] = '5GB',
    safe_serialization: bool = True,
    variant: Optional[str] = None,
    save_peft_format: bool = True,
    tags: List[str] = None,
    temporary_location: str = '_unsloth_temporary_saved_buffers',
    maximum_memory_usage: float = 0.75,
)
Source:   
def unsloth_generic_save_pretrained_merged(
    self,
    save_directory       : Union[str, os.PathLike],
    tokenizer            = None,
    save_method          : str = "merged_16bit", # ["lora", "merged_16bit", "merged_4bit"]
    push_to_hub          : bool = False,
    token                : Optional[Union[str, bool]] = None,
    is_m

In [ ]:
model.save_pretrained_merged("../Model/merged_16b", save_method = "merged_16bit",)

Unsloth: You're not saving a tokenizer as well?
You can do it separately via `tokenizer.save_pretrained(...)`


Unsloth: Merging weights into 16bit:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  50%|█████     | 1/2 [02:46<02:46, 166.56s/it]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [04:56<00:00, 148.17s/it]


In [1]:
from vllm import LLM, SamplingParams
# llm = LLM(model="../Model/merged_16b")
llm = LLM(model="unsloth/gemma-3-4b-it-unsloth-bnb-4bit")


# # Step 2: Prepare pre-tokenized input
# input_text = "Hello, world!"
# input_tokens = tokenizer.encode(input_text, return_tensors="pt").tolist()[0]  # Get token IDs as a list
# print(f"Input tokens: {input_tokens}")

# # Step 3: Configure sampling parameters
# sampling_params = SamplingParams(
#     temperature=0.7,
#     max_tokens=20,
#     # Ensure we don't skip tokenization checks unnecessarily
# )



INFO 04-05 11:55:31 [__init__.py:239] Automatically detected platform cuda.
INFO 04-05 11:55:35 [config.py:585] This model supports multiple tasks: {'classify', 'reward', 'score', 'generate', 'embed'}. Defaulting to 'generate'.
WARNING 04-05 11:55:36 [config.py:664] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 04-05 11:55:36 [arg_utils.py:1854] --quantization bitsandbytes is not supported by the V1 Engine. Falling back to V0. 
WARNING 04-05 11:55:36 [arg_utils.py:1741] The model has a long context length (131072). This may causeOOM during the initial memory profiling phase, or result in low performance due to small KV cache size. Consider setting --max-model-len to a smaller value.
INFO 04-05 11:55:36 [llm_engine.py:241] Initializing a V0 LLM engine (v0.8.2) with config: model='unsloth/gemma-3-4b-it-unsloth-bnb-4bit', speculative_config=None, tokenizer='unsloth/gemma-3-4b-it-unsloth-bnb-4bit', skip_tokenizer_init=False

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


AssertionError: 

In [4]:
sampling_params = SamplingParams(
    n = 5,
    temperature=0.7,
    max_tokens=20,
    # Ensure we don't skip tokenization checks unnecessarily
)

In [3]:
input_tokens

[2, 31414, 6, 232, 328]

In [13]:
# Step 4: Pass token IDs directly to vLLM
# Note: vLLM's high-level `generate` API expects text, so we use a list of token IDs with a custom approach
# We'll wrap it in a way that mimics the expected input structure
request_input = [input_tokens]  # vLLM expects a list of inputs (batch)
outputs = llm.generate(prompt_token_ids=request_input,  # Use prompt_token_ids instead of text prompts
                        sampling_params=sampling_params,
                        use_tqdm=False  # Disable progress bar for clarity
                    )

# Step 5: Extract output token IDs
for output in outputs:
    for i in output.outputs:
        output_tokens = i.token_ids  # Raw token IDs
        output_text = tokenizer.decode(output_tokens)  # Decode for human-readable output
        print(f"Output tokens: {output_tokens}")
        print(f"Decoded output: {output_text}")

# Optional: If you only want tokens and no decoding, just use output_tokens directly

Output tokens: [50118, 50118, 100, 216, 1085, 59, 5, 232, 6, 53, 38, 348, 57, 1207, 11, 1261, 13, 5, 94, 891]
Decoded output: 

I know nothing about the world, but I've been living in Florida for the last couple
Output tokens: [50118, 2678, 6, 20280, 4, 50118, 100, 437, 888, 269, 6661, 7, 28, 259, 4, 50118, 100, 2035, 95, 11]
Decoded output: 
oh, hi.
I'm actually really sorry to be here.
I arrived just in
Output tokens: [38, 437, 6661, 6, 38, 348, 57, 447, 15, 5, 754, 14, 38, 524, 41, 2370, 5385, 4, 38, 437]
Decoded output:  I'm sorry, I've been working on the fact that I am an English speaker. I'm
Output tokens: [50118, 50118, 31414, 328, 38, 524, 5, 29085, 328, 50118, 50118, 2264, 16, 5, 766, 9, 127, 5059, 116, 50118]
Decoded output: 

Hello! I am the Witch!

What is the name of my blog?

Output tokens: [50118, 30086, 89, 4, 50118, 31414, 4, 50118, 100, 437, 6661, 4, 50118, 100, 21, 10, 410, 628, 4, 50118]
Decoded output: 
Hi there.
Hello.
I'm sorry.
I was a little late.



In [11]:
for i in output.outputs:
    print(i.token_ids)

[50118, 50118, 1106, 47, 32, 2509, 11, 3736, 84, 3221, 6, 2540, 27754, 50, 5124, 4, 370, 40, 28, 441]
[50118, 30086, 6, 232, 328, 50118, 31414, 6, 232, 328, 50118, 31414, 6, 232, 328, 50118, 31414, 6, 232, 328]
[50118, 50118, 250, 621, 54, 34, 57, 1207, 11, 5, 382, 13, 262, 107, 122, 6, 3026, 162, 14, 51]
[38, 437, 10, 664, 621, 11, 5, 382, 54, 6138, 7, 1504, 36, 100, 437, 4959, 11, 1777, 4170, 322]
[38, 437, 95, 667, 7, 6136, 10, 936, 6, 8, 38, 437, 45, 686, 99, 7, 109, 4, 38, 33]
